In [22]:
import numpy as np
import math

# Linear Algebra

### LU Decomposition

In [23]:
# def pivot(matrix):
#     order = matrix.shape[0]
#     for line in range(order-1):
#         if (matrix[line][line]) = 0

def get_matrix_decomposition_step(matrix, step_index):        
        order = matrix.shape[0]
        pivot = matrix[step_index, step_index]
        
        if (pivot == 0):
            raise ValueError('Pivot is 0!', matrix, step_index)
            return
            # pivotting missing here
#           return get_matrix_decomposition_step(pivot(matrix), step_index)
        
        lower = np.identity(order)
        upper = np.identity(order)
        
        for line_index in range (step_index + 1, order):
            element = matrix[line_index, step_index] / pivot
            
            lower[line_index, step_index] = element
            upper[line_index, step_index] = - element
            
        return lower, upper
    
def lu_decompose(matrix):
    order = matrix.shape[0]
    upper = matrix
    upper_combination = np.identity(order)
    lower = np.identity(order)
    
    for step_index in range (order - 1):
        lower_step, upper_step = get_matrix_decomposition_step(upper, step_index)
        upper_combination = upper_step @ upper_combination
        lower = lower_step@lower
        upper = upper_step@upper

    return lower, upper, upper_combination


### Cholesky decomposition

In [24]:
def check_simetric_positive_defined(matrix):
    order = matrix.shape[0]
    
    for i in range(order):
        if matrix[i,i] < 0:
            return false
        
        for j in range(order):
            if matrix[i,j] != matrix[j,i]:
                return False
    
    return True

In [25]:
def cholesky_decompose(A):
    order = A.shape[0]

    L = np.zeros((order, order))

    for i in range(order):
        for k in range(i+1):
            tmp_sum = sum(L[i,j] * L[k,j] for j in range(k))
            
            if (i == k):
                L[i,k] = math.sqrt(A[i,i] - tmp_sum)
            else:
                L[i,k] = (1.0 / L[k,k] * (A[i,k] - tmp_sum))
    return L

### Gauss elimination

In [26]:
def gauss_elimination(matrix):
    _, upper, upper_combination = lu_decompose(matrix)
    return upper, upper_combination

### Solve AX = B by gauss elimination

In [27]:
def solve_equation_gauss_elimination(A, B):
    order = A.shape[0]
    last_index = order-1

    A, combination_matrix = gauss_elimination(A)
    B = combination_matrix @ B
    
    X = np.zeros(order)
    
    X[last_index] = B[last_index] / A[last_index, last_index]
        
    for x_index in range(last_index - 1, -1, -1): # from n-1 to 0
        backwards_sum = 0
        for sum_index in range(last_index, x_index, -1):
            backwards_sum += A[x_index, sum_index] * X[sum_index] 
        
        X[x_index] = (B[x_index] - backwards_sum) / A[x_index, x_index]
        
    return X

### Determinant

In [28]:
def determinant(matrix):
    # pivotting missing here
    
    order = matrix.shape[0]
    upper, lower, _ = lu_decompose(matrix)
    lower_det = 1
    upper_det = 1
    
    for index in range(order):
        lower_det = lower[index, index] * lower_det
        upper_det = upper[index, index] * upper_det
        
    return lower_det * upper_det

### Solve AX = B by Jacobi method

In [29]:
def check_diagonal_dominance(A):
    order = A.shape[0]
    for diagonal_index in range(order):
        row_sum = 0
        column_sum = 0
        diagonal_item = A[diagonal_index, diagonal_index]
        
        for item_index in range(order):
            row_sum += A[diagonal_index, item_index]
            column_sum += A[item_index, diagonal_index]
        
        if (diagonal_item < row_sum or diagonal_item < column_sum):
            return False
        
        return True

def solve_equation_jacobi(A, B, tolerance=0.000001):
    order = A.shape[0]
    residue = tolerance + 1
    X = np.ones(order)
    
    if (not check_diagonal_dominance(A)):
        return -1
    
    while(residue > tolerance):
        currentX = np.ones(order)
        
        for x_index in range (order):
            subtrahend = 0
            
            for sum_index in range (order):
                if (sum_index == x_index):
                    continue
                subtrahend += A[x_index, sum_index] * X[sum_index]
            
            currentX[x_index] = (B[x_index] - subtrahend) / A[x_index, x_index]
        
        residue = np.linalg.norm(currentX - X, ord=2) / np.linalg.norm(currentX, ord=2)
        X = currentX
    
    return X
        

### Solve AX = B by Gauss-seidel method

In [30]:
def solve_equation_gauss_seidel(A, B, tolerance=0.000001):
    order = A.shape[0]
    residue = tolerance + 1
    X = np.ones(order)
    
    if (not check_simetric_positive_defined(A) and not check_diagonal_dominance(A)):
        return -1
    
    while(residue > tolerance):
        currentX = np.ones(order)
        
        for i in range (order):
            subtrahend = 0
            
            for j in range (i):
                subtrahend += A[i, j] * currentX[j]
    
            for j in range (i + 1, order):
                subtrahend += A[i, j] * X[j]

            currentX[i] = (B[i] - subtrahend) / A[i, i]
        
        residue = np.linalg.norm(currentX - X, ord=2) / np.linalg.norm(currentX, ord=2)
        X = currentX
    
    return X

# Tests

In [31]:
def match(expected, result, acceptClose=False):
    if (type(expected) != type(result)):
        raise TypeError(f'❌ expected {expected}, got: {result}',)
        return
    
    if (type(expected) != np.ndarray and expected != result):
        raise ValueError(f'❌ expected {expected}, got: {result}',)
        return 
    
    if (not acceptClose and not np.array_equal(expected,result)):
        raise ValueError(f'❌ expected {expected}, got: {result}')
        return 
    
    if (acceptClose and not np.allclose(expected,result, rtol=0.1)):
        raise ValueError(f'❌ expected {expected}, got: {result}')
        return 
        
    return print("✅ pass!")

### Cholesky Decompostion

In [32]:
def solve_cholesky_decompose():
    A = np.array(
        [[1, .2, .4],
        [.2, 1, .5],
        [.4, .5, 1]]
    )

    result = cholesky_decompose(A)
    
    expected = np.array([[1,  0, 0],[.2,.98,0],[.4,.43,.81]])
    
    match(expected, result, acceptClose=True)

### Solve AX = B by gauss elimination

In [33]:
def solve_3_by_3_gauss():
    A = np.array(
        [[1, 2, 2],
        [4, 4, 2],
        [4, 6, 4]]
    )

    B = np.array([ 3, 6, 10])

    result = solve_equation_gauss_elimination(A, B)
    expected = np.array([-1.,  3., -1.])
    
    match(expected, result)

def solve_2_by_2_pivot_gauss(): 
    A = np.array([[1,1],[1,1]])

    B = np.array([2,2])

    result = solve_equation_gauss_elimination(A, B)
    expected = np.array([1, 1])
    
    match(expected, result)

### Solve AX = B by jacobi

In [34]:
def solve_3_by_3_jacobi():
    A = np.array(
        [[3, -1, -1],
        [-1, +3, -1],
        [-1, -1, +3]]
    )

    B = np.array([1, 2, 1])

    result = solve_equation_jacobi(A, B)
    expected = np.array([1.25,  1.5, 1.25])
    
    match(expected, result, acceptClose=True)
    
def does_not_converge_jacobi():
    A = np.array(
        [[1, 2, 2],
        [4, 4, 2],
        [4, 6, 4]]
    )

    B = np.array([3, 6, 10])

    result = solve_equation_jacobi(A, B)
    expected = -1
    
    match(expected, result, acceptClose=True)


In [35]:
def solve_3_by_3_gauss_seidel():
    A = np.array(
        [[3, -1, -1],
        [-1, +3, -1],
        [-1, -1, +3]]
    )

    B = np.array([1, 2, 1])

    result = solve_equation_gauss_seidel(A, B)
    expected = np.array([1.25,  1.5, 1.25])
    
    match(expected, result, acceptClose=True)
    
def does_not_converge_gauss_seidel():
    A = np.array(
        [[1, 2, 2],
        [4, 4, 2],
        [4, 6, 4]]
    )

    B = np.array([3, 6, 10])

    result = solve_equation_gauss_seidel(A, B)
    expected = -1
    
    match(expected, result, acceptClose=True)

In [36]:
solve_cholesky_decompose()
solve_3_by_3_gauss_seidel()
does_not_converge_gauss_seidel()
solve_3_by_3_gauss()
# solve_2_by_2_pivot_gauss()

✅ pass!
✅ pass!
✅ pass!
✅ pass!
